In [ ]:
#
# Project:
#      PyTorch Dojo (https://github.com/wo3kie/ml-dojo)
#
# Author:
#      Lukasz Czerwinski (https://www.lukaszczerwinski.pl/)
#

$$ H(X) = \sum_{e \in \mathcal{E}} P(X=e) \, log_2 \left( \frac{1}{P(X=e)} \right) $$
$$ \\[2em]$$
$$ H(X) = \text{How many bits on average} $$
$$ \\[2em]$$
$$ \text{Example} $$
$$ X = \Bigg[ \frac{1}{2}, \frac{1}{4}, \frac{1}{8}, \cdots \Bigg] $$
$$ H(X) = \frac{1}{2} \log_2(2) + \frac{1}{4} \log_2(4) + \frac{1}{8} \log_2(8) + \cdots = $$
$$ \frac{1}{2} \cdot 1 + \frac{1}{4} \cdot 2 + \frac{1}{8} \cdot 3 + \cdots = $$
$$ 2 $$
$$ \text{1 bit frequently (50\%), 2 bits rarely (25\%), 3 bits even more rarely (12.5\%), ... } $$

In [ ]:
from torch import long, manual_seed, randn, Tensor

import import_ipynb
import common # type: ignore
from common import assert_eq, assert_ne, T # type: ignore

import cross_entropy # type: ignore


def entropy(iterable, count=False, norm=False):
    """
    Calculates the entropy of samples when `count` is True, or the entropy of samples otherwise.
    """

    iterable = T(iterable)

    if count == True:
        iterable = common.count(iterable)[1]

    if (count == True) or (norm == True):
        iterable = iterable / iterable.sum()

    return cross_entropy._cross_entropy_d(iterable, iterable)
    

def test_entropy_1():
    #
    # The entropy of the certain distribution (0% / 100%) is 0.
    #

    assert_eq(entropy([1]), 0.0, atol=0.01)
    # assert_eq(entropy([0]), 0.0, atol=0.01) --- IGNORE ---

    assert_eq(entropy([0, 1]), 0.0, atol=0.01)
    assert_eq(entropy([1, 0]), 0.0, atol=0.01)


def test_entropy_2():
    #
    # The entropy of a distribution is maximum when all elements are equal,
    # and in that case it is equal to log2(n) where n is the number of elements in the distribution.
    #

    assert_eq(entropy([1/2, 1/2]), 1.0, atol=0.01)
    assert_eq(entropy([1/4, 1/4, 1/4, 1/4]), 2.0, atol=0.01)
    assert_eq(entropy([1/8, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8]), 3.0, atol=0.01)

    #
    # By smoothing the distribution we can increase the entropy, 
    # but it cannot exceed log2(n) where n is the number of elements in the distribution.
    #

    assert_eq(entropy([1/2, 1/2]), 1.000, atol=0.01)
    assert_eq(entropy([1/2, 1/4, 1/4]), 1.500, atol=0.01)
    assert_eq(entropy([1/2, 1/4, 1/8, 1/8]), 1.750, atol=0.01)
    assert_eq(entropy([1/4, 1/4, 1/4, 1/8, 1/8]), 2.250, atol=0.01)
    assert_eq(entropy([1/4, 1/4, 1/8, 1/8, 1/8, 1/8]), 2.500, atol=0.01)
    assert_eq(entropy([1/4, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8]), 2.750, atol=0.01)
    assert_eq(entropy([1/8, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8]), 3.0, atol=0.01)


def test_entropy_3():
    #
    # Geometric sequence, entropy(1/2, 1/4, 1/8, ...) -> 2.0
    #

    assert_eq(entropy([1/2, 1/4, 1/8, 1/16, 1/32, 1/64, 1/128, 1/256, 1/512, 1/1024], norm=True), 1.988, atol=0.01)


def test_entropy_4():
    #
    # Heavy-tail distributions have lower entropy than uniform distribution.
    #

    assert_eq(entropy([3/12, 3/12, 3/12, 3/12]), 2.0, atol=0.01)
    assert_eq(entropy([4/12, 2/12, 3/12, 3/12]), 1.959, atol=0.01)
    assert_eq(entropy([5/12, 1/12, 3/12, 3/12]), 1.825, atol=0.01)
    assert_eq(entropy([6/12, 1/12, 2/12, 3/12]), 1.723, atol=0.01)
    assert_eq(entropy([7/12, 1/12, 1/12, 3/12]), 1.551, atol=0.01)
    assert_eq(entropy([8/12, 1/12, 1/12, 2/12]), 1.418, atol=0.01)
    assert_eq(entropy([9/12, 1/12, 1/12, 1/12]), 1.207, atol=0.01)


def test_entropy_5():
    #
    # Zipf's distribution has dominant element (decrease entropy),
    # and the tail of the distribution is long (increase entropy),
    # therefore the entropy of Zipf's distribution is high but not maximum.
    #

    assert_eq(entropy([1/4, 1/4, 1/4, 1/4], norm=True), 2.0, atol=0.01)
    assert_eq(entropy([1/1, 1/2, 1/3, 1/4], norm=True), 1.7924, atol=0.01)

    assert_eq(entropy([1/8, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8], norm=True), 3.0, atol=0.01)
    assert_eq(entropy([1/1, 1/2, 1/3, 1/4, 1/5, 1/6, 1/7, 1/8], norm=True), 2.6197, atol=0.01)


def test_entropy_6():
    #
    # Uniform distribution, the higher standard deviation, the (logarithmically) higher entropy.
    #
    
    normal = lambda size, std: (std * randn(size)).round().to(long)

    manual_seed(0)
    assert_eq(entropy(normal(size=1024,   std=1.0), count=True), 2.117, atol=0.01)
    assert_eq(entropy(normal(size=1024,   std=5.0), count=True), 4.356, atol=0.01)
    assert_eq(entropy(normal(size=1024,  std=10.0), count=True), 5.258, atol=0.01)
    assert_eq(entropy(normal(size=1024,  std=25.0), count=True), 6.512, atol=0.01)
    assert_eq(entropy(normal(size=1024,  std=50.0), count=True), 7.473, atol=0.01)
    assert_eq(entropy(normal(size=1024, std=100.0), count=True), 8.278, atol=0.01)


def test_entropy_7():
    entropy_from_string = lambda s: entropy(list(map(ord, s)), count=True)

    assert_eq(entropy_from_string("aaaaaaaaaaaaaaaa"), 0.000, atol=0.01)
    assert_eq(entropy_from_string("aaaaaaaaaaaaaabb"), 0.543, atol=0.01)
    assert_eq(entropy_from_string("aaaaaaaaaaaabbbb"), 0.811, atol=0.01)
    assert_eq(entropy_from_string("aaaaaaaaaabbbbbb"), 0.954, atol=0.01)
    assert_eq(entropy_from_string("aaaaaaaabbbbbbbb"), 1.000, atol=0.01)

    
#
# it execute this code when running %run command, but not when using import_ipynb
#

if __name__ == "__main__":
    test_entropy_1()
    test_entropy_2()
    test_entropy_3()
    test_entropy_4()
    test_entropy_5()
    test_entropy_6()
    test_entropy_7()
